2019 Census Data for Selected Variables - Baltimore City

In [1]:
#pip install us

In [2]:
# pip install censusgeocode
# pip install censusdata

In [3]:
#conda install -c conda-forge cenpy

In [4]:
#conda update -n base -c defaults conda


In [5]:
# From https://cenpy-devs.github.io/cenpy/:
# Cenpy (pronounced sen-pie) is a package that automatically discovers US Census Bureau API endpoints and exposes them to Python in a consistent fashion. 
# It also provides easy-to-use access to certain well-used data products, like the American Community Survey (ACS) and 2010 Decennial Census.
#pip install cenpy

In [6]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [7]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata 
import censusgeocode as cg
import cenpy
import gmaps
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Census & gmaps API Keys
from config import (api_key, gkey)
c = Census(api_key, year=2018)

# Configure gmaps
#gmaps.configure(api_key=gkey)

C:\Users\Jenkir\anaconda3\envs\PythonData\lib\site-packages\libpysal\weights\util.py:23: UserWarning: geopandas not available. Some functionality will be disabled.
  warn("geopandas not available. Some functionality will be disabled.")


In [8]:
# Set this to false if you're trying to do this without an internet connection
# and data which would have been fetched from an API query will be read from cached files instead
INTERNET_IS_WORKING = True

if INTERNET_IS_WORKING:
    con = cenpy.remote.APIConnection('ACSDT5Y2019')
    variables = con.variables
else:
    variables = pd.read_csv('data/ACSDT5Y2019_variables.csv',index_col='Unnamed: 0')

# No matter which product you use, a cenpy APIConnection can show you the variables 
# which it can retrieve for you. They come back as a pandas DataFrame.
print(f"ACSDT5Y2019 provides {len(variables)} variables.") # how many are there?
variables.head()

ACSDT5Y2019 provides 27080 variables.


,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN,NaN,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN,NaN,NaN
ucgid,Uniform Census Geography Identifier clause,Census API Geography Specification,ucgid,N/A,0,True,True,NaN,NaN
B24022_060E,Estimate!!Total:!!Female:!!Service occupations...,SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE P...,int,B24022,0,NaN,NaN,"B24022_060EA,B24022_060M,B24022_060MA",NaN
B19001B_014E,"Estimate!!Total:!!$100,000 to $124,999",HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 201...,int,B19001B,0,NaN,NaN,"B19001B_014EA,B19001B_014M,B19001B_014MA",NaN


In [9]:
# Comments and code in this block are from https://github.com/censusreporter/nicar20-advanced-census-python/blob/master/workshop.ipynb:
# will use (Nicar20) as citation for the above site from here on out
# values for 'group' are ACS table IDs; 
# for this data, when it's N/A, it's for other kinds of API variables.
# so let's leave those out:
short_vars = variables[~(variables['group'] == 'N/A')] 

# So now this is an easy way to get a list of all of the table IDs and their titles
short_vars[['group', 'concept']].drop_duplicates().sort_values('group').head(10) 

,group,concept
B01001_012E,B01001,SEX BY AGE
B01001A_002E,B01001A,SEX BY AGE (WHITE ALONE)
B01001B_029E,B01001B,SEX BY AGE (BLACK OR AFRICAN AMERICAN ALONE)
B01001C_008E,B01001C,SEX BY AGE (AMERICAN INDIAN AND ALASKA NATIVE ...
B01001D_008E,B01001D,SEX BY AGE (ASIAN ALONE)
B01001E_013E,B01001E,SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ...
B01001F_001E,B01001F,SEX BY AGE (SOME OTHER RACE ALONE)
B01001G_022E,B01001G,SEX BY AGE (TWO OR MORE RACES)
B01001H_001E,B01001H,"SEX BY AGE (WHITE ALONE, NOT HISPANIC OR LATINO)"
B01001I_027E,B01001I,SEX BY AGE (HISPANIC OR LATINO)


In [10]:
# (From Nicar20)
# Use when you know which group but still need specific API variable codes
# Attributes column shows related variables you can request. The one that ends with M is the margin of error, and since we want to be responsible when we aggregate data, we'll be sure to aggregate the error as well. 
#The other two, which end with A are "annotations." 
short_vars[short_vars['group'] == 'B01001'][['label','attributes']].sort_index() 

,label,attributes
B01001_001E,Estimate!!Total:,"B01001_001EA,B01001_001M,B01001_001MA"
B01001_002E,Estimate!!Total:!!Male:,"B01001_002EA,B01001_002M,B01001_002MA"
B01001_003E,Estimate!!Total:!!Male:!!Under 5 years,"B01001_003EA,B01001_003M,B01001_003MA"
B01001_004E,Estimate!!Total:!!Male:!!5 to 9 years,"B01001_004EA,B01001_004M,B01001_004MA"
B01001_005E,Estimate!!Total:!!Male:!!10 to 14 years,"B01001_005EA,B01001_005M,B01001_005MA"
B01001_006E,Estimate!!Total:!!Male:!!15 to 17 years,"B01001_006EA,B01001_006M,B01001_006MA"
B01001_007E,Estimate!!Total:!!Male:!!18 and 19 years,"B01001_007EA,B01001_007M,B01001_007MA"
B01001_008E,Estimate!!Total:!!Male:!!20 years,"B01001_008EA,B01001_008M,B01001_008MA"
B01001_009E,Estimate!!Total:!!Male:!!21 years,"B01001_009EA,B01001_009M,B01001_009MA"
B01001_010E,Estimate!!Total:!!Male:!!22 to 24 years,"B01001_010EA,B01001_010M,B01001_010MA"


In [11]:
# search function
#sample = censusdata.search('acs5', 2019, 'concept', 'age ')
#print(sample)

#  to see list of all tables in the ACS5
# c.acs5.tables()

In [12]:
censusdata.printtable(censusdata.censustable('acs1', 2018, 'B08604'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B08604_001E  | WORKER POPULATION FOR WORKPLAC | !! Estimate Total                                        | int  
-------------------------------------------------------------------------------------------------------------------


In [13]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
# The state FIPS code for MD is 24 and the FIPS code for Balt City is 510
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",                                                            
                          "B15003_017E",
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_008E",
                          "B03001_003E",
                          "B25008_002E",
                          "B28007_001E",
                          "B25003_002E",                                    
                          "B25003_003E"),               
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
census_pd = pd.DataFrame(census_data)
census_pd      

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_003E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract
0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,121.0,611.0,413.0,25.0,...,1967.0,26.0,62.0,508.0,1506.0,156.0,652.0,24,510,190100
1,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,117.0,918.0,179.0,155.0,...,1024.0,155.0,233.0,485.0,1481.0,200.0,498.0,24,510,190200
2,"Census Tract 2201, Baltimore city, Maryland",75652.0,4335.0,33.9,63475.0,449.0,81.0,2745.0,313.0,1266.0,...,592.0,197.0,305.0,1887.0,3962.0,928.0,1336.0,24,510,220100
3,"Census Tract 2303, Baltimore city, Maryland",106806.0,1431.0,30.5,61426.0,120.0,12.0,1172.0,114.0,536.0,...,75.0,17.0,48.0,692.0,1379.0,331.0,399.0,24,510,230300
4,"Census Tract 2502.07, Baltimore city, Maryland",34081.0,2308.0,31.1,19115.0,700.0,102.0,861.0,483.0,153.0,...,2041.0,107.0,129.0,563.0,1708.0,150.0,667.0,24,510,250207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,"Census Tract 1507.01, Baltimore city, Maryland",47109.0,1573.0,46.7,23432.0,311.0,59.0,738.0,300.0,174.0,...,1442.0,59.0,12.0,1057.0,1384.0,369.0,220.0,24,510,150701
196,"Census Tract 1503, Baltimore city, Maryland",38194.0,2301.0,36.5,22417.0,545.0,139.0,896.0,520.0,97.0,...,2258.0,10.0,8.0,1199.0,1873.0,394.0,309.0,24,510,150300
197,"Census Tract 1507.02, Baltimore city, Maryland",41471.0,2164.0,41.1,25251.0,522.0,125.0,869.0,511.0,297.0,...,2086.0,48.0,2.0,1289.0,1832.0,565.0,261.0,24,510,150702
198,"Census Tract 1511, Baltimore city, Maryland",51827.0,6482.0,41.8,30893.0,1137.0,393.0,2585.0,1245.0,483.0,...,6354.0,88.0,0.0,4591.0,5237.0,1797.0,817.0,24,510,151100


In [14]:
age_data = c.acs5.state_county_tract(("NAME", "B01001_003E",                                    
                          "B01001_004E",
                          "B01001_005E",
                          "B01001_006E",
                          "B01001_007E",
                          "B01001_008E",
                          "B01001_009E",
                          "B01001_010E",
                          "B01001_011E",
                          "B01001_012E",
                          "B01001_013E",
                          "B01001_014E",
                          "B01001_015E",
                          "B01001_016E",            
                          "B01001_017E",
                          "B01001_018E",
                          "B01001_019E",
                          "B01001_020E",
                          "B01001_021E",
                          "B01001_022E",
                          "B01001_023E",
                          "B01001_024E",
                          "B01001_025E",
                          
                          "B01001_027E",
                          "B01001_028E",
                          "B01001_029E",
                          "B01001_030E",
                          "B01001_031E",
                          "B01001_032E",
                          "B01001_033E",
                          "B01001_034E",
                          "B01001_035E",
                          "B01001_036E",
                          "B01001_037E",
                          "B01001_038E",
                          "B01001_039E",            
                          "B01001_040E",
                          "B01001_041E",
                          "B01001_042E",
                          "B01001_043E",
                          "B01001_044E",
                          "B01001_045E",
                          "B01001_046E",
                          "B01001_047E",
                          "B01001_048E",
                          "B01001_049E"),          
                                     
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
age_pd = pd.DataFrame(age_data)
age_pd.head()      

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_043E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract
0,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,8.0,24.0,7.0,25.0,53.0,38.0,27.0,24,510,190100
1,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,6.0,9.0,32.0,12.0,26.0,0.0,0.0,24,510,190200
2,"Census Tract 2201, Baltimore city, Maryland",43.0,50.0,169.0,0.0,0.0,0.0,29.0,62.0,387.0,...,15.0,12.0,173.0,81.0,111.0,47.0,61.0,24,510,220100
3,"Census Tract 2303, Baltimore city, Maryland",0.0,0.0,0.0,3.0,0.0,0.0,0.0,105.0,231.0,...,9.0,13.0,10.0,7.0,4.0,0.0,15.0,24,510,230300
4,"Census Tract 2502.07, Baltimore city, Maryland",144.0,51.0,100.0,20.0,55.0,21.0,14.0,30.0,83.0,...,54.0,32.0,22.0,58.0,8.0,37.0,23.0,24,510,250207


In [15]:
# Sum columns of age groups that are under 15 years old for male and female and add new column "Pop. under 15 years"
columns_under15 = ["B01001_003E",                                    
                   "B01001_004E",
                   "B01001_005E",
                   "B01001_027E",
                   "B01001_028E",
                   "B01001_029E"]
age_pd['Pop. under 15 years']= age_pd[columns_under15].sum(axis=1)
age_pd.head()

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. under 15 years
0,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,24.0,7.0,25.0,53.0,38.0,27.0,24,510,190100,595.0
1,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,9.0,32.0,12.0,26.0,0.0,0.0,24,510,190200,349.0
2,"Census Tract 2201, Baltimore city, Maryland",43.0,50.0,169.0,0.0,0.0,0.0,29.0,62.0,387.0,...,12.0,173.0,81.0,111.0,47.0,61.0,24,510,220100,313.0
3,"Census Tract 2303, Baltimore city, Maryland",0.0,0.0,0.0,3.0,0.0,0.0,0.0,105.0,231.0,...,13.0,10.0,7.0,4.0,0.0,15.0,24,510,230300,13.0
4,"Census Tract 2502.07, Baltimore city, Maryland",144.0,51.0,100.0,20.0,55.0,21.0,14.0,30.0,83.0,...,32.0,22.0,58.0,8.0,37.0,23.0,24,510,250207,592.0


In [16]:
# Sum columns of age groups that are 15-74 years old (working age) for male and female and add new column "Pop. under 15 years"
columns_working_age = ["B01001_006E",                                    
                   "B01001_007E",
                   "B01001_008E",
                   "B01001_009E",
                   "B01001_010E",
                   "B01001_011E",
                   "B01001_012E",   
                   "B01001_013E",
                   "B01001_014E",    
                   "B01001_015E",    
                   "B01001_016E",   
                   "B01001_017E", 
                   "B01001_018E",    
                   "B01001_019E",    
                   "B01001_020E",    
                   "B01001_021E",
                   "B01001_022E",
                   "B01001_030E",
                   "B01001_031E",
                   "B01001_032E",
                   "B01001_033E",
                   "B01001_034E",
                   "B01001_035E",   
                   "B01001_036E",
                   "B01001_037E",    
                   "B01001_038E",    
                   "B01001_039E",   
                   "B01001_040E", 
                   "B01001_041E",    
                   "B01001_042E",    
                   "B01001_043E",    
                   "B01001_044E",
                   "B01001_045E",    
                   "B01001_046E"]         
age_pd['Pop. working age']= age_pd[columns_working_age].sum(axis=1)
age_pd.head()                
              
               


,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. under 15 years,Pop. working age
0,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,7.0,25.0,53.0,38.0,27.0,24,510,190100,595.0,1368.0
1,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,32.0,12.0,26.0,0.0,0.0,24,510,190200,349.0,1499.0
2,"Census Tract 2201, Baltimore city, Maryland",43.0,50.0,169.0,0.0,0.0,0.0,29.0,62.0,387.0,...,173.0,81.0,111.0,47.0,61.0,24,510,220100,313.0,3602.0
3,"Census Tract 2303, Baltimore city, Maryland",0.0,0.0,0.0,3.0,0.0,0.0,0.0,105.0,231.0,...,10.0,7.0,4.0,0.0,15.0,24,510,230300,13.0,1395.0
4,"Census Tract 2502.07, Baltimore city, Maryland",144.0,51.0,100.0,20.0,55.0,21.0,14.0,30.0,83.0,...,22.0,58.0,8.0,37.0,23.0,24,510,250207,592.0,1635.0


In [17]:
# Sum columns of age groups that are 75+ years old for male and female and add new column "Pop. over 74 years"
columns_senior = ["B01001_023E",                                    
                  "B01001_024E",
                  "B01001_025E",
                  "B01001_047E",                                    
                  "B01001_048E",
                  "B01001_049E"]               
                          
age_pd['Pop. over 74 years']= age_pd[columns_senior].sum(axis=1)
age_pd.head()                           

,NAME,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,...,B01001_046E,B01001_047E,B01001_048E,B01001_049E,state,county,tract,Pop. under 15 years,Pop. working age,Pop. over 74 years
0,"Census Tract 1901, Baltimore city, Maryland",55.0,66.0,43.0,32.0,23.0,0.0,0.0,0.0,75.0,...,25.0,53.0,38.0,27.0,24,510,190100,595.0,1368.0,168.0
1,"Census Tract 1902, Baltimore city, Maryland",64.0,155.0,22.0,54.0,17.0,0.0,0.0,14.0,74.0,...,12.0,26.0,0.0,0.0,24,510,190200,349.0,1499.0,38.0
2,"Census Tract 2201, Baltimore city, Maryland",43.0,50.0,169.0,0.0,0.0,0.0,29.0,62.0,387.0,...,81.0,111.0,47.0,61.0,24,510,220100,313.0,3602.0,420.0
3,"Census Tract 2303, Baltimore city, Maryland",0.0,0.0,0.0,3.0,0.0,0.0,0.0,105.0,231.0,...,7.0,4.0,0.0,15.0,24,510,230300,13.0,1395.0,23.0
4,"Census Tract 2502.07, Baltimore city, Maryland",144.0,51.0,100.0,20.0,55.0,21.0,14.0,30.0,83.0,...,58.0,8.0,37.0,23.0,24,510,250207,592.0,1635.0,81.0


In [18]:
age_final = age_pd[[ "tract", "Pop. under 15 years", "Pop. working age", "Pop. over 74 years"]]
age_final.head()

,tract,Pop. under 15 years,Pop. working age,Pop. over 74 years
0,190100,595.0,1368.0,168.0
1,190200,349.0,1499.0,38.0
2,220100,313.0,3602.0,420.0
3,230300,13.0,1395.0,23.0
4,250207,592.0,1635.0,81.0


In [19]:
# did not add in daytime population - will use ESRI business analyst for this 
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state_county(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          county_fips = "510") 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

,NAME,B08604_001E,state,county
0,"Baltimore city, Maryland",382638.0,24,510


In [20]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract #
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census_pd["GEOID"] = census_pd['state'] + census_pd['county'] + census_pd['tract']
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract,GEOID
0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,121.0,611.0,413.0,25.0,...,26.0,62.0,508.0,1506.0,156.0,652.0,24,510,190100,24510190100
1,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,117.0,918.0,179.0,155.0,...,155.0,233.0,485.0,1481.0,200.0,498.0,24,510,190200,24510190200
2,"Census Tract 2201, Baltimore city, Maryland",75652.0,4335.0,33.9,63475.0,449.0,81.0,2745.0,313.0,1266.0,...,197.0,305.0,1887.0,3962.0,928.0,1336.0,24,510,220100,24510220100
3,"Census Tract 2303, Baltimore city, Maryland",106806.0,1431.0,30.5,61426.0,120.0,12.0,1172.0,114.0,536.0,...,17.0,48.0,692.0,1379.0,331.0,399.0,24,510,230300,24510230300
4,"Census Tract 2502.07, Baltimore city, Maryland",34081.0,2308.0,31.1,19115.0,700.0,102.0,861.0,483.0,153.0,...,107.0,129.0,563.0,1708.0,150.0,667.0,24,510,250207,24510250207


In [21]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

Number of rows, columns:  (200, 22)


In [22]:
# combine age data with rest of census data
census_joined = pd.concat([age_final, census_pd], axis="columns")
census_joined.head()

,tract,Pop. under 15 years,Pop. working age,Pop. over 74 years,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,...,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract,GEOID
0,190100,595.0,1368.0,168.0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,...,26.0,62.0,508.0,1506.0,156.0,652.0,24,510,190100,24510190100
1,190200,349.0,1499.0,38.0,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,...,155.0,233.0,485.0,1481.0,200.0,498.0,24,510,190200,24510190200
2,220100,313.0,3602.0,420.0,"Census Tract 2201, Baltimore city, Maryland",75652.0,4335.0,33.9,63475.0,449.0,...,197.0,305.0,1887.0,3962.0,928.0,1336.0,24,510,220100,24510220100
3,230300,13.0,1395.0,23.0,"Census Tract 2303, Baltimore city, Maryland",106806.0,1431.0,30.5,61426.0,120.0,...,17.0,48.0,692.0,1379.0,331.0,399.0,24,510,230300,24510230300
4,250207,592.0,1635.0,81.0,"Census Tract 2502.07, Baltimore city, Maryland",34081.0,2308.0,31.1,19115.0,700.0,...,107.0,129.0,563.0,1708.0,150.0,667.0,24,510,250207,24510250207


In [23]:
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's more easily understandable
census_joined = census_joined.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median age",
                                      "B19013_001E": "Median household income",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_008E": "Pop. 2 or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "Under 15 years": "Pop. under 15 years",
                                      "B28007_001E": "Labor force status by presence of computer/ internet in HH",
                                      "NAME": "Name", "state": "State", "GEOID": "GEOID"})

# Add a new column for poverty rate (Poverty Count / Population)
census_joined["Poverty rate"] = 100 * \
    census_joined["Poverty count"].astype(
        int) / census_joined["Population"].astype(int)


# Add a new column for unemployment rate (Employment Count / Population)
census_joined["Unemployment rate"] = 100 * \
    census_joined["Unemployment count"].astype(
        int) / census_joined["Population"].astype(int)
census_joined.head()

,tract,Pop. under 15 years,Pop. working age,Pop. over 74 years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,Total pop. in occupied housing units by tenure,Labor force status by presence of computer/ internet in HH,Total owner-occupied units,Total renter-occupied units,State,county,tract,GEOID,Poverty rate,Unemployment rate
0,190100,595.0,1368.0,168.0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,...,508.0,1506.0,156.0,652.0,24,510,190100,24510190100,50.070389,5.678085
1,190200,349.0,1499.0,38.0,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,...,485.0,1481.0,200.0,498.0,24,510,190200,24510190200,31.336161,6.203606
2,220100,313.0,3602.0,420.0,"Census Tract 2201, Baltimore city, Maryland",75652.0,4335.0,33.9,63475.0,449.0,...,1887.0,3962.0,928.0,1336.0,24,510,220100,24510220100,10.357555,1.868512
3,230300,13.0,1395.0,23.0,"Census Tract 2303, Baltimore city, Maryland",106806.0,1431.0,30.5,61426.0,120.0,...,692.0,1379.0,331.0,399.0,24,510,230300,24510230300,8.385744,0.838574
4,250207,592.0,1635.0,81.0,"Census Tract 2502.07, Baltimore city, Maryland",34081.0,2308.0,31.1,19115.0,700.0,...,563.0,1708.0,150.0,667.0,24,510,250207,24510250207,30.329289,4.419411


In [24]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_joined.shape)

Number of rows, columns:  (200, 28)


In [25]:
# Add in home ownership rate (# owner-occupied units / # of occupied housing units)
# sum 2 columns: total owner-occupied units + total renter-occupied units to create additional column "Total occupied units" 
sum_column = census_joined['Total owner-occupied units'] + census_joined['Total renter-occupied units']
census_joined["Total occupied units"] = sum_column

In [26]:
census_joined["Home ownership rate"] = 100 * \
    census_joined["Total owner-occupied units"].astype(
        int) / census_joined["Total occupied units"].astype(
        int) 

census_joined.head()

,tract,Pop. under 15 years,Pop. working age,Pop. over 74 years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,Total owner-occupied units,Total renter-occupied units,State,county,tract,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,190100,595.0,1368.0,168.0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,...,156.0,652.0,24,510,190100,24510190100,50.070389,5.678085,808.0,19.306931
1,190200,349.0,1499.0,38.0,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,...,200.0,498.0,24,510,190200,24510190200,31.336161,6.203606,698.0,28.653295
2,220100,313.0,3602.0,420.0,"Census Tract 2201, Baltimore city, Maryland",75652.0,4335.0,33.9,63475.0,449.0,...,928.0,1336.0,24,510,220100,24510220100,10.357555,1.868512,2264.0,40.989399
3,230300,13.0,1395.0,23.0,"Census Tract 2303, Baltimore city, Maryland",106806.0,1431.0,30.5,61426.0,120.0,...,331.0,399.0,24,510,230300,24510230300,8.385744,0.838574,730.0,45.342466
4,250207,592.0,1635.0,81.0,"Census Tract 2502.07, Baltimore city, Maryland",34081.0,2308.0,31.1,19115.0,700.0,...,150.0,667.0,24,510,250207,24510250207,30.329289,4.419411,817.0,18.359853


In [27]:
# round the home ownership rate to one decimal point; using "float" instead of "int" because want to use decimal points
census_joined["Home ownership rate"] = census_joined["Home ownership rate"].astype(float).round(1)


In [28]:
census_joined["Poverty rate"] = census_joined["Poverty rate"].astype(float).round(1)


In [29]:
census_joined["Unemployment rate"] = census_joined["Unemployment rate"].astype(float).round(1)
census_joined.head()

,tract,Pop. under 15 years,Pop. working age,Pop. over 74 years,Name,Median household income,Population,Median age,Per capita income,Poverty count,...,Total owner-occupied units,Total renter-occupied units,State,county,tract,GEOID,Poverty rate,Unemployment rate,Total occupied units,Home ownership rate
0,190100,595.0,1368.0,168.0,"Census Tract 1901, Baltimore city, Maryland",21630.0,2131.0,34.9,17518.0,1067.0,...,156.0,652.0,24,510,190100,24510190100,50.1,5.7,808.0,19.3
1,190200,349.0,1499.0,38.0,"Census Tract 1902, Baltimore city, Maryland",32500.0,1886.0,34.4,22552.0,591.0,...,200.0,498.0,24,510,190200,24510190200,31.3,6.2,698.0,28.7
2,220100,313.0,3602.0,420.0,"Census Tract 2201, Baltimore city, Maryland",75652.0,4335.0,33.9,63475.0,449.0,...,928.0,1336.0,24,510,220100,24510220100,10.4,1.9,2264.0,41.0
3,230300,13.0,1395.0,23.0,"Census Tract 2303, Baltimore city, Maryland",106806.0,1431.0,30.5,61426.0,120.0,...,331.0,399.0,24,510,230300,24510230300,8.4,0.8,730.0,45.3
4,250207,592.0,1635.0,81.0,"Census Tract 2502.07, Baltimore city, Maryland",34081.0,2308.0,31.1,19115.0,700.0,...,150.0,667.0,24,510,250207,24510250207,30.3,4.4,817.0,18.4


In [30]:
# Calculate population density  see: https://www.census.gov/quickfacts/fact/note/US/LND110210
# density is expressed as "population per square mile(kilometer)"
# Divide total population (or # of housing units)/ by land area of the entity measured in square miles

In [31]:
# Change order of columns in DataFrame by using double brackets
census_joined = census_joined[["State", "Name", "GEOID", "Population", "Median household income",
                       "Per capita income", "Poverty count", "Poverty rate", "Unemployment rate", 
                       "# employed, age 16+", "Unemployment count",
                      "# persons age 25+ graduated high school", "# persons age 25+ with Bachelor's degree",
                      "Median age","Pop. white", "Pop. Black", "Pop. 2 or more races", "Pop. Hispanic origin", 
                      "Total pop. in occupied housing units by tenure", "Total owner-occupied units", "Total renter-occupied units",
                      "Pop. under 15 years", "Pop. working age", "Pop. over 74 years",         
                      "Labor force status by presence of computer/ internet in HH"]]

census_joined.head()

,State,Name,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. under 15 years,Pop. working age,Pop. over 74 years,Labor force status by presence of computer/ internet in HH
0,24,"Census Tract 1901, Baltimore city, Maryland",24510190100,2131.0,21630.0,17518.0,1067.0,50.1,5.7,611.0,...,1967.0,26.0,62.0,508.0,156.0,652.0,595.0,1368.0,168.0,1506.0
1,24,"Census Tract 1902, Baltimore city, Maryland",24510190200,1886.0,32500.0,22552.0,591.0,31.3,6.2,918.0,...,1024.0,155.0,233.0,485.0,200.0,498.0,349.0,1499.0,38.0,1481.0
2,24,"Census Tract 2201, Baltimore city, Maryland",24510220100,4335.0,75652.0,63475.0,449.0,10.4,1.9,2745.0,...,592.0,197.0,305.0,1887.0,928.0,1336.0,313.0,3602.0,420.0,3962.0
3,24,"Census Tract 2303, Baltimore city, Maryland",24510230300,1431.0,106806.0,61426.0,120.0,8.4,0.8,1172.0,...,75.0,17.0,48.0,692.0,331.0,399.0,13.0,1395.0,23.0,1379.0
4,24,"Census Tract 2502.07, Baltimore city, Maryland",24510250207,2308.0,34081.0,19115.0,700.0,30.3,4.4,861.0,...,2041.0,107.0,129.0,563.0,150.0,667.0,592.0,1635.0,81.0,1708.0


In [32]:
census_joined.count()

State                                                         200
Name                                                          200
GEOID                                                         200
Population                                                    200
Median household income                                       200
Per capita income                                             200
Poverty count                                                 200
Poverty rate                                                  199
Unemployment rate                                             199
# employed, age 16+                                           200
Unemployment count                                            200
# persons age 25+ graduated high school                       200
# persons age 25+ with Bachelor's degree                      200
Median age                                                    200
Pop. white                                                    200
Pop. Black

In [33]:
# Split the "Name" column into 3 separate columns: "Census_Tract", "County", "State"
census_joined[['Census_Tract', "County", "State"]]= census_joined['Name'].str.split(",", n=3, expand=True)
census_joined.head()

,State,Name,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. under 15 years,Pop. working age,Pop. over 74 years,Labor force status by presence of computer/ internet in HH,Census_Tract,County
0,Maryland,"Census Tract 1901, Baltimore city, Maryland",24510190100,2131.0,21630.0,17518.0,1067.0,50.1,5.7,611.0,...,62.0,508.0,156.0,652.0,595.0,1368.0,168.0,1506.0,Census Tract 1901,Baltimore city
1,Maryland,"Census Tract 1902, Baltimore city, Maryland",24510190200,1886.0,32500.0,22552.0,591.0,31.3,6.2,918.0,...,233.0,485.0,200.0,498.0,349.0,1499.0,38.0,1481.0,Census Tract 1902,Baltimore city
2,Maryland,"Census Tract 2201, Baltimore city, Maryland",24510220100,4335.0,75652.0,63475.0,449.0,10.4,1.9,2745.0,...,305.0,1887.0,928.0,1336.0,313.0,3602.0,420.0,3962.0,Census Tract 2201,Baltimore city
3,Maryland,"Census Tract 2303, Baltimore city, Maryland",24510230300,1431.0,106806.0,61426.0,120.0,8.4,0.8,1172.0,...,48.0,692.0,331.0,399.0,13.0,1395.0,23.0,1379.0,Census Tract 2303,Baltimore city
4,Maryland,"Census Tract 2502.07, Baltimore city, Maryland",24510250207,2308.0,34081.0,19115.0,700.0,30.3,4.4,861.0,...,129.0,563.0,150.0,667.0,592.0,1635.0,81.0,1708.0,Census Tract 2502.07,Baltimore city


In [34]:
# Remove "State" column because it's understood we are looking at MD data
census_joined = census_joined.drop(["State"], axis=1)


In [35]:
# list the columns in the census_pd dataframe
census_joined.columns

Index(['Name', 'GEOID', 'Population', 'Median household income',
       'Per capita income', 'Poverty count', 'Poverty rate',
       'Unemployment rate', '# employed, age 16+', 'Unemployment count',
       '# persons age 25+ graduated high school',
       '# persons age 25+ with Bachelor's degree', 'Median age', 'Pop. white',
       'Pop. Black', 'Pop. 2 or more races', 'Pop. Hispanic origin',
       'Total pop. in occupied housing units by tenure',
       'Total owner-occupied units', 'Total renter-occupied units',
       'Pop. under 15 years', 'Pop. working age', 'Pop. over 74 years',
       'Labor force status by presence of computer/ internet in HH',
       'Census_Tract', 'County'],
      dtype='object')

In [36]:
# Create new column "Census_Tract" and remove the text "Census Tract" from the values in that column (to make calculations easier)
census_joined["Census_Tract"] = census_joined['Census_Tract'].str.replace('Census Tract', "") 


In [37]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_joined["Census_Tract"].unique())
tract_count

200

In [38]:
census_final = census_joined.drop(["Name"], axis=1)
census_final.head()

,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,# persons age 25+ graduated high school,...,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Pop. under 15 years,Pop. working age,Pop. over 74 years,Labor force status by presence of computer/ internet in HH,Census_Tract,County
0,24510190100,2131.0,21630.0,17518.0,1067.0,50.1,5.7,611.0,121.0,413.0,...,62.0,508.0,156.0,652.0,595.0,1368.0,168.0,1506.0,1901,Baltimore city
1,24510190200,1886.0,32500.0,22552.0,591.0,31.3,6.2,918.0,117.0,179.0,...,233.0,485.0,200.0,498.0,349.0,1499.0,38.0,1481.0,1902,Baltimore city
2,24510220100,4335.0,75652.0,63475.0,449.0,10.4,1.9,2745.0,81.0,313.0,...,305.0,1887.0,928.0,1336.0,313.0,3602.0,420.0,3962.0,2201,Baltimore city
3,24510230300,1431.0,106806.0,61426.0,120.0,8.4,0.8,1172.0,12.0,114.0,...,48.0,692.0,331.0,399.0,13.0,1395.0,23.0,1379.0,2303,Baltimore city
4,24510250207,2308.0,34081.0,19115.0,700.0,30.3,4.4,861.0,102.0,483.0,...,129.0,563.0,150.0,667.0,592.0,1635.0,81.0,1708.0,2502.07,Baltimore city


In [39]:
# Export file as a CSV, without the Pandas index, but with the header
#census_pd.to_csv("CommCorr_Census_Stats_2018.csv", index = False, header=True)